In [9]:
import ccxt
import config
from datetime import datetime, timedelta
import numpy as np

# Initialize Binance US exchange
binance_us = ccxt.binanceus({
    'apiKey': config.BINANCE_US_API_KEY,
    'secret': config.BINANCE_US_SECRET_KEY,
})

def fetch_ohlcv_last_minutes(pair, minutes=5, timeframe='1m'):
    """Fetch the last 'minutes' of OHLCV data for the given 'pair'."""
    since = binance_us.milliseconds() - minutes * 60 * 1000  # 'minutes' ago from now
    ohlcv = binance_us.fetch_ohlcv(pair, timeframe, since, limit=minutes)
    return np.array(ohlcv)

def determine_market_condition(ohlcv):
    """Determine if the market is bullish, bearish, or sideways."""
    open_prices, close_prices = ohlcv[:, 1], ohlcv[:, 4]  # Open and close prices
    avg_open, avg_close = np.mean(open_prices), np.mean(close_prices)
    
    if avg_close > avg_open * 1.01:  # Modify thresholds as needed
        return 'bullish'
    elif avg_close < avg_open * 0.99:
        return 'bearish'
    else:
        return 'sideways'

def predict_pancakeswap_round(pair='BNB/USD', prediction_interval=5):
    """
    Decide whether to bet bull or bear on the next PancakeSwap prediction round.
    This function should be executed 1 minute before the round closes.
    """
    # Fetch historical data to determine market condition
    ohlcv = fetch_ohlcv_last_minutes(pair, prediction_interval)
    market_condition = determine_market_condition(ohlcv)
    
    # Strategy decision based on market condition
    if market_condition == 'bullish':
        return 'Bet on Bull'
    elif market_condition == 'bearish':
        return 'Bet on Bear'
    else:
        # Implement additional logic for sideways market
        # For example, check the volume, implement range-bound strategies, etc.
        return 'No Bet - Sideways Market'

# Example usage (This line should be scheduled to run 1 minute before PancakeSwap round closes)
suggestion = predict_pancakeswap_round()
print(f"Suggested Action: {suggestion}")

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed